In [14]:
from deepcave.runs.converters.smac3v2 import SMAC3v2Run
from deepcave.plugins.hyperparameter.importances import Importances
from matplotlib import pyplot as plt
import numpy as np

In [15]:
def evaluate_feature_importance(path_to_smac_run:str, objective_id:int):
    run = SMAC3v2Run.from_path(path_to_smac_run)
    objective_id = run.get_objective_ids()[objective_id]
    budget_ids = run.get_budget_ids()

    # Instantiate the plugin
    plugin = Importances()
    inputs = plugin.generate_inputs(
        hyperparameter_names=list(run.configspace.keys()),
        objective_id=objective_id,
        budget_ids=budget_ids,
        method="global",
        n_hps=20,
        n_trees=10,
    )
    # Note: Filter variables are not considered.
    outputs = plugin.generate_outputs(run, inputs)

    return outputs

In [16]:
outputs = evaluate_feature_importance("smac3_output/grow_once/grow_depth_once_results/0", 0)[0]
outputs

{'momentum_reset': (0.0009069916513087389,
  2.4361728254430334e-06,
  0.0009069916513087389,
  2.4361728254430334e-06),
 'n_epochs': (0.0024102075655980703,
  1.1777329206976594e-05,
  0.0024102075655980703,
  1.1777329206976594e-05),
 'batch_size': (0.003430852423300266,
  6.950879933253302e-05,
  0.003430852423300266,
  6.950879933253302e-05),
 'ent_coef': (0.2417147252053644,
  0.012888333446559892,
  0.2417147252053644,
  0.012888333446559892),
 'learning_rate': (0.6906852512353009,
  0.018048200066951143,
  0.6906852512353009,
  0.018048200066951143)}

In [17]:
outputs = sorted(outputs.items(), key=lambda x: x[1][0], reverse=True)
outputs

[('learning_rate',
  (0.6906852512353009,
   0.018048200066951143,
   0.6906852512353009,
   0.018048200066951143)),
 ('ent_coef',
  (0.2417147252053644,
   0.012888333446559892,
   0.2417147252053644,
   0.012888333446559892)),
 ('batch_size',
  (0.003430852423300266,
   6.950879933253302e-05,
   0.003430852423300266,
   6.950879933253302e-05)),
 ('n_epochs',
  (0.0024102075655980703,
   1.1777329206976594e-05,
   0.0024102075655980703,
   1.1777329206976594e-05)),
 ('momentum_reset',
  (0.0009069916513087389,
   2.4361728254430334e-06,
   0.0009069916513087389,
   2.4361728254430334e-06))]

In [19]:
importances = [item[1][0] for item in outputs]
errors = [item[1][1] for item in outputs]
features = ["Learning Rate", "Entropy Coefficient", "Batch Size", "n Epochs", "Momentum Reset"]

cmap = plt.get_cmap("tab10")  # Get the tab10 colormap
colors = cmap(np.arange(len(features))) 

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(features, importances, color=colors, yerr=errors, capsize=15, error_kw={'elinewidth': 2, 'capthick': 2, 'ecolor': 'black'})
ax.set_xlabel("Features")
ax.set_ylabel("Importance")
ax.set_title("Feature Importances; Grow Width Once", fontsize=18, fontweight="bold")
ax.legend(title="Model Type", fontsize=12, title_fontsize=14, loc="center", bbox_to_anchor=(0.5, -0.16), ncol=3)
ax.grid(True, linestyle="--", alpha=0.7)

plt.savefig("plotting/minihack/grow_depth_once/grow_depth_once_feature_importances.png", bbox_inches='tight')

/tmp/ipykernel_387990/2258036385.py:14: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(title="Model Type", fontsize=12, title_fontsize=14, loc="center", bbox_to_anchor=(0.5, -0.16), ncol=3)
